In [71]:
import math
import numpy as np 
    
def replicas(x, y, L): 
    """
    Function to generate replicas of a single particle.
    
    Parameters
    ==========
    x, y : Position.
    L : Side of the squared arena.
    """    
    xr = np.zeros(9)
    yr = np.zeros(9)

    for i in range(3):
        for j in range(3):
            xr[3 * i + j] = x + (j - 1) * L
            yr[3 * i + j] = y + (i - 1) * L
    
    return xr, yr

def pbc(x, y, L):
    """
    Function to enforce periodic boundary conditions on the positions.
    
    Parameters
    ==========
    x, y : Position.
    L : Side of the squared arena.
    """   
    
    outside_left = np.where(x < - L / 2)[0]
    x[outside_left] = x[outside_left] + L

    outside_right = np.where(x > L / 2)[0]
    x[outside_right] = x[outside_right] - L

    outside_up = np.where(y > L / 2)[0]
    y[outside_up] = y[outside_up] - L

    outside_down = np.where(y < - L / 2)[0]
    y[outside_down] = y[outside_down] + L
    
    return x, y
from functools import reduce

def interaction(x, y, theta, Rf, L):
    """
    Function to calculate the orientation at the next time step.
    
    Parameters
    ==========
    x, y : Positions.
    theta : Orientations.
    Rf : Flocking radius.
    L : Dimension of the squared arena.
    s : Discrete steps.
    """
    
    N = np.size(x)

    theta_next = np.zeros(N)
    
    # Preselect what particles are closer than Rf to the boundaries.
    replicas_needed = reduce( 
        np.union1d, (
            np.where(y + Rf > L / 2)[0], 
            np.where(y - Rf < - L / 2)[0],
            np.where(x + Rf > L / 2)[0],
            np.where(x - Rf > - L / 2)[0]
        )
    )

    for j in range(N):
        # Check if replicas are needed to find the nearest neighbours.
        if np.size(np.where(replicas_needed == j)[0]):
            # Use replicas.
            xr, yr = replicas(x[j], y[j], L)
            nn = []
            for nr in range(9):
                dist2 = (x - xr[nr]) ** 2 + (y - yr[nr]) ** 2 
                nn = np.union1d(nn, np.where(dist2 <= Rf ** 2)[0])
        else:
            dist2 = (x - x[j]) ** 2 + (y - y[j]) ** 2 
            nn = np.where(dist2 <= Rf ** 2)[0]
        
        # The list of nearest neighbours is set.
        nn = nn.astype(int)
        
        # Circular average.
        av_sin_theta = np.mean(np.sin(theta[nn]))
        av_cos_theta = np.mean(np.cos(theta[nn]))
        
        theta_next[j] = np.arctan2(av_sin_theta, av_cos_theta)
                   
    return theta_next

def global_alignment(theta):
    """
    Function to calculate the global alignment coefficient.
    
    Parameters
    ==========
    theta : Orientations.
    """
    
    N = np.size(theta)
    
    global_direction_x = np.sum(np.sin(theta))
    global_direction_y = np.sum(np.cos(theta))
        
    psi = np.sqrt(global_direction_x ** 2 + global_direction_y ** 2) / N
    
                   
    return psi

from scipy.spatial import Voronoi, voronoi_plot_2d

def area_polygon(vertices):
    """
    Function to calculate the area of a Voronoi region given its vertices.
    
    Parameters
    ==========
    vertices : Coordinates (array, 2 dimensional).
    """    
    
    N, dim = vertices.shape
    
    # dim is 2.
    # Vertices are listed consecutively.
    
    A = 0
    
    for i in range(N-1):
        # Below is the formula of the area of a triangle given the vertices.
        A += np.abs(
            vertices[- 1, 0] * (vertices[i, 1] - vertices[i + 1, 1]) +
            vertices[i, 0] * (vertices[i + 1, 1] - vertices[- 1, 1]) +
            vertices[i + 1, 0] * (vertices[- 1, 1] - vertices[i, 1])
        )
    
    A *= 0.5
    
    return A


def global_clustering(x, y, Rf, L):
    """
    Function to calculate the global alignment coefficient.
    
    Parameters
    ==========
    x, y : Positions.
    Rf : Flocking radius.
    L : Dimension of the squared arena.
    """
    
    N = np.size(x)
    
    # Use the replicas of all points to calculate Voronoi for 
    # a more precise estimate.
    points = np.zeros([9 * N, 2])

    for i in range(3):
        for j in range(3):
            s = 3 * i + j
            points[s * N:(s + 1) * N, 0] = x + (j - 1) * L
            points[s * N:(s + 1) * N, 1] = y + (i - 1) * L

    # The format of points is the one needed by Voronoi.
    # points[:, 0] contains the x coordinates
    # points[:, 1] contains the y coordinates
   
    vor = Voronoi(points)     
    '''
    vertices = vor.vertices  # Voronoi vertices.
    regions = vor.regions  # Region list. 
    # regions[i]: list of the vertices indices for region i.
    # If -1 is listed: the region is open (includes point at infinity).
    point_region = vor.point_region  # Region associated to input point.
    '''
   
    # Consider only regions of original set of points (no replicas).
    list_regions = vor.point_region[4 * N:5 * N]
    
    c = 0

    for i in list_regions:
        indices = vor.regions[i]
        # print(f'indices = {indices}')
        if len(indices) > 0:
            if np.size(np.where(np.array(indices) == -1)[0]) == 0:
                # Region is finite.
                # Calculate area.
                A = area_polygon(vor.vertices[indices,:])
                if A < np.pi * Rf ** 2:
                    c += 1
                    
    c = c / N
                   
    return c

def predator_behavior(x, y, xp, yp, thetap, L, N_pred, vp_pred, dt):

    """
    Update predator positions and orientations based on nearest prey.

    Parameters:
    x, y : Prey positions.
    xp, yp : Predator positions.
    thetap : Predator orientations.
    L : Arena size.
    """ 

    for j in range(N_pred):
        # Find nearest prey
        distances = np.sqrt((x - xp[j]) ** 2 + (y - yp[j]) ** 2)
        nearest_prey = np.argmin(distances)

        # Update predator orientation to move towards the nearest prey
        dx = x[nearest_prey] - xp[j]
        dy = y[nearest_prey] - yp[j]
        thetap[j] = np.arctan2(dy, dx)

    # Update predator positions
    xp += vp_pred * np.cos(thetap) * dt
    yp += vp_pred * np.sin(thetap) * dt

    # Apply periodic boundary conditions
    xp, yp = pbc(xp, yp, L)

    return xp, yp, thetap

def fleeing_behavior(x, y, theta, xp, yp, Rflee, L):
    """
    Update prey orientations to include fleeing behavior from nearby predators.

    Parameters:
    x, y : ndarray
        Prey positions.
    theta : ndarray
        Prey orientations.
    xp, yp : ndarray
        Predator positions.
    Rflee : float
        Fleeing radius within which prey reacts to predators.
    L : float
        Arena size.

    Returns:
    ndarray
        Updated prey orientations after considering fleeing behavior.
    """
    N_prey = len(x)
    N_pred = len(xp)

    # Iterate through each prey
    for i in range(N_prey):
        # Calculate distances to all predators
        distances = np.sqrt((xp - x[i]) ** 2 + (yp - y[i]) ** 2)
        close_predators = np.where(distances < Rflee)[0]

        if len(close_predators) >= 1:
            # Compute fleeing direction (away from closest predators)
            dx = np.sum(x[i] - xp[close_predators])
            dy = np.sum(y[i] - yp[close_predators])
            flee_angle = np.arctan2(dy, dx)

            # Blend fleeing direction with flocking direction
            theta[i] = 0.7 * flee_angle + 0.3 * theta[i]

    return theta

def remove_captured_prey(x,y,nx, ny, theta, ntheta, particles, velocities, xp, yp, Rcapture,N_part, canvas):
    """
    Remove prey that are within the capture radius of any predator.

    Parameters:
    x, y : ndarray
        Prey positions.
    theta : ndarray
        Prey orientations.
    particles : list
        Tkinter objects for prey visualization.
    velocities : list
        Tkinter objects for velocity visualization.
    xp, yp : ndarray
        Predator positions.
    Rcapture : float
        Radius within which predators catch prey.
    canvas : Tkinter canvas
        Canvas object for visualization.

    Returns:
    Updated x, y, theta, particles, velocities after removing captured prey.
    """
    N_prey = len(x)
    to_remove = []

    for i in range(N_prey):
        # Check if prey `i` is within the capture radius of any predator
        distances = np.sqrt((xp - x[i]) ** 2 + (yp - y[i]) ** 2)
        if np.any(distances < Rcapture):
            to_remove.append(i)  # Mark prey for removal

    # Remove captured prey
    for idx in sorted(to_remove, reverse=True):  # Remove in reverse to avoid index shifts
        canvas.delete(particles[idx])  # Remove prey visualization
        canvas.delete(velocities[idx])  # Remove velocity visualization
        particles.pop(idx)
        velocities.pop(idx)
        x = np.delete(x, idx)
        y = np.delete(y, idx)
        nx = np.delete(nx, idx)
        ny = np.delete(ny, idx)
        theta = np.delete(theta, idx)
        ntheta = np.delete(ntheta, idx)
        N_prey -= 1
    return x, y, nx, ny, theta,ntheta, particles, velocities, N_prey



In [72]:
N_part = 200  # Number of particles.
L = 100  # Dimension of the squared arena.
v = 2  # Speed.
Rf = 10  # Flocking radius.
eta = 0.2  # Noise.  Try values: 0.01, 0.3, 1.0, 2 * np.pi
dt = 0.1  # Time step.

# Initialization.

# Random position.
x = (np.random.rand(N_part) - 0.5) * L  # in [-L/2, L/2]
y = (np.random.rand(N_part) - 0.5) * L  # in [-L/2, L/2]

# Random orientation.
theta = 2 * (np.random.rand(N_part) - 0.5) * np.pi  # in [-pi, pi]


In [73]:
import time
# from scipy.constants import Boltzmann as kB 
from tkinter import *

window_size = 600

rp = 0.5  # Plotting radius of a particle.
vp = 1  # Length of the arrow indicating the velocity direction.
line_width = 1  # Width of the arrow line.

N_skip = 1

N_pred = 5  # Example: 5 predators

# Speed and sensing radius of predators
vp_pred = 5  # Speed of predators
Rp_pred = 10  # Sensing radius of predators
Rflee = 5  # Fleeing radius
Rcapture = 1 # Capture radius

# Initialize predator positions and orientations
xp = (np.random.rand(N_pred) - 0.5) * L  # in [-L/2, L/2]
yp = (np.random.rand(N_pred) - 0.5) * L  # in [-L/2, L/2]
thetap = 2 * (np.random.rand(N_pred) - 0.5) * np.pi  # in [-pi, pi]

tk = Tk()
tk.geometry(f'{window_size + 20}x{window_size + 20}')
tk.configure(background='#000000')

canvas = Canvas(tk, background='#ECECEC')  # Generate animation window 
tk.attributes('-topmost', 0)
canvas.place(x=10, y=10, height=window_size, width=window_size)

particles = []
for j in range(N_part):
    particles.append(
        canvas.create_oval(
            (x[j] - rp) / L * window_size + window_size / 2, 
            (y[j] - rp) / L * window_size + window_size / 2,
            (x[j] + rp) / L * window_size + window_size / 2, 
            (y[j] + rp) / L * window_size + window_size / 2,
            outline='#FF0000', 
            fill='#FF0000',
        )
    )

predators = []
for j in range(N_pred):
    predators.append(
        canvas.create_oval(
            (xp[j] - rp) / L * window_size + window_size / 2,
            (yp[j] - rp) / L * window_size + window_size / 2,
            (xp[j] + rp) / L * window_size + window_size / 2,
            (yp[j] + rp) / L * window_size + window_size / 2,
            outline="#0000FF",  # Blue for predators
            fill="#0000FF",
        )
    )

velocities = []
for j in range(N_part):
    velocities.append(
        canvas.create_line(
            x[j] / L * window_size + window_size / 2, 
            y[j] / L * window_size + window_size / 2,
            (x[j] + vp * np.cos(theta[j])) / L * window_size + window_size / 2, 
            (y[j] + vp * np.cos(theta[j])) / L * window_size + window_size / 2,
            width=line_width
        )
    )

step = 0

def stop_loop(event):
    global running
    running = False

tk.bind("<Escape>", stop_loop)  # Bind the Escape key to stop the loop.
running = True  # Flag to control the loop.

while running:
    
    dtheta = eta * (np.random.rand(N_part) - 0.5) * dt
    ntheta = interaction(x, y, theta, Rf, L) + dtheta
    ntheta = fleeing_behavior(x, y, ntheta, xp, yp, Rflee, L)
    ntheta += dtheta
    nx = x + v * np.cos(ntheta)
    ny = y + v * np.sin(ntheta)
    
    nx, ny = pbc(nx, ny, L)

    xp, yp, thetap = predator_behavior(nx, ny, xp, yp, thetap, L, N_pred, vp_pred, dt)

    thetap = fleeing_behavior(xp, yp, thetap, xp, yp, Rflee, L)
    
    x, y, nx, ny, theta, ntheta, particles, velocities, N_part = remove_captured_prey(
        x, y, nx, ny, theta, ntheta, particles, velocities, xp, yp, Rcapture, N_part, canvas
    )

    # Update animation frame.
    if step % N_skip == 0:   

        for j, particle in enumerate(particles):
            canvas.coords(
                particle,
                (nx[j] - rp) / L * window_size + window_size / 2,
                (ny[j] - rp) / L * window_size + window_size / 2,
                (nx[j] + rp) / L * window_size + window_size / 2,
                (ny[j] + rp) / L * window_size + window_size / 2,
            )
        for j, predator in enumerate(predators):
            canvas.coords(
                predator,
                (xp[j] - rp) / L * window_size + window_size / 2,
                (yp[j] - rp) / L * window_size + window_size / 2,
                (xp[j] + rp) / L * window_size + window_size / 2,
                (yp[j] + rp) / L * window_size + window_size / 2,
            )
                    
        for j, velocity in enumerate(velocities):
            canvas.coords(
                velocity,
                nx[j] / L * window_size + window_size / 2,
                ny[j] / L * window_size + window_size / 2,
                (nx[j] + vp * np.cos(ntheta[j])) / L * window_size + window_size / 2,
                (ny[j] + vp * np.sin(ntheta[j])) / L * window_size + window_size / 2,
            )
                    
        tk.title(f'Time {step * dt:.1f} - Iteration {step}')
        tk.update_idletasks()
        tk.update()
        time.sleep(.1)  # Increase to slow down the simulation.    

    step += 1
    x[:] = nx[:]
    y[:] = ny[:]
    theta[:] = ntheta[:]  

tk.update_idletasks()
tk.update()
tk.mainloop()  # Release animation handle (close window to finish).

TclError: invalid command name ".!canvas"